##__<div style="text-align: center;">Vibration Data Analyze utilizing Apache Spark</div>__
####<div style="text-align: center;">_Steven Tsuang, Research Assistant, National Taiwan University._</div>
####This program is writen for conducting vibration.  The program featuring Apache Spark, a innovative distributing computer platform.

###__Introduction__
####This is a case study of vibration condition in a hi-tech factory. It is 20-year 10-story-high moment frame structure located in Hsinchu science park (_figure 1_), Tawan. To analysis the vibration serviceability of the structure, a series of sensors was deployed in the 5<sup>th</sup> and the 9<sup>th</sup> and the basement.  
####Uning these data, we will go though a series of structure analyze to discover the pattern of vibration condition. A abridge demo version of the analyze is also offered considering the 


###__Step 1 FTP connection__
####The is now operating in situ. not distubing the data aquisition process.


In [1]:
#log in the ftp server in situ.
from ftplib import FTP
ftp=FTP("218.210.127.247")
ftp.login("root","cccccccc")

#Retrieving the file name for each file.
ftpDirList=ftp.nlst()
print(ftpDirList[0:5])
print(len(ftpDirList))

['2015-01-18-10-53-45_131072.csv', '2015-01-18-11-04-40_131072.csv', '2015-01-18-11-15-35_131072.csv', '2015-01-18-11-37-27_131072.csv', '2015-01-18-11-48-22_131072.csv']
39255


####Date to July 31st, there are already 35126 files on the FTP server.  The underlie code retrieve data imformation from its file name.

In [2]:
#Extract date time from file name
import datetime
import re
from datetime import timedelta

def extractDateTime(filename):
    pattern="(\d+)-(\d+)-(\d+)-(\d+)-(\d+)-(\d+)_\d+.csv"
    match = re.search(pattern,filename)
    return datetime.datetime(int(match.group(1)),
                             int(match.group(2)),
                             int(match.group(3)),
                             int(match.group(4)),
                             int(match.group(5)),
                             int(match.group(6)))
def extractDataNum(filename):
    pattern="\d+-\d+-\d+-\d+-\d+-\d+_(\d+).csv"
    match = re.search(pattern,filename)
    return int(match.group(1))

def getStartTime(endtime,datanum,samplefreq):
    return endtime-timedelta(seconds=datanum/samplefreq)

def retriExactTime(filename,targettime):
    endtime=extractDateTime(filename)
    datanum=extractDataNum(filename)
    starttime=getStartTime(endtime,datanum,200)
    if (endtime>targettime) & (starttime<targettime):
        return True
    else: return False

endtime=extractDateTime('2015-01-18-11-37-27_131072.csv')
datanum=extractDataNum('2015-01-18-11-37-27_131072.csv')
starttime=getStartTime(endtime,datanum,200)

testTargetTime=datetime.datetime(2015,1,18,11,26,27)
print retriExactTime('2015-01-18-11-37-27_131072.csv',testTargetTime)

print(starttime,endtime,datanum)

try:
    print(extractDataNum("I'm-a-troll-you-are-a_fool.csv"))
except:
    print('YOU SHOULD NOT PASS')
    

False
(datetime.datetime(2015, 1, 18, 11, 26, 32), datetime.datetime(2015, 1, 18, 11, 37, 27), 131072)
YOU SHOULD NOT PASS


In [5]:
#nameListRDD=sc.parallelize(ftpDirList,4)
targetTime=datetime.datetime(2015,8,8,5,26,30)
#2015-07-15-11-09-31_131072
periodStart=datetime.datetime(2015,8,19,10,8,20)
pattern="(\d+)-(\d+)-(\d+)-(\d+)-(\d+)-(\d+)_\d+.csv"
cleanedDirList=filter(lambda n:re.search(pattern,n),ftpDirList)
targetName=filter(lambda n:retriExactTime(n,targetTime),cleanedDirList)[0]
downloadList=filter(lambda n:extractDateTime(n)>periodStart,cleanedDirList)

#targetName=nameListRDD.filter(lambda n:retriExactTime(n,targetTime)).collect()
print targetName
#20150323-182458
print downloadList[0],len(downloadList)

2015-08-08-05-26-32_131072.csv
2015-08-19-10-19-14_131072.csv 1644


####First, we retrieve a file for testing. 

In [4]:
import sys
import ftplib
import time

ftp=FTP("218.210.127.247")
ftp.login("root","cccccccc")

txt=open('2015-08-08-05-26-32_131072.csv','w')

def gettext(ftp, filename, outfile=None):
    # fetch a text file
    if outfile is None:
        outfile = sys.stdout
    # use a lambda to add newlines to the lines read from the server
    ftp.retrlines("RETR " + filename, lambda s, w=outfile.write: w(s+"\n"))

start=time.time()
gettext(ftp,'2015-03-23-18-14-03_131072.csv',txt)
end=time.time()

print 'process time '+str(end-start)+' seconds/per file'
txt.close()

process time 13.6420710087 seconds/per file


In [7]:
tryList=downloadList[0:5]
print tryList

['2015-07-15-11-09-31_131072.csv', '2015-07-15-11-20-27_131072.csv', '2015-07-15-11-31-24_131072.csv', '2015-07-15-11-42-19_131072.csv', '2015-07-15-11-53-15_131072.csv']


In [7]:
def retrieveData(ftp, filelist, save_path):
    for n in filelist:
        try:
            completeName = os.path.join(save_path,n)
            csv=open(completeName,'w')
            start=time.time()
            ftp.retrlines("RETR " + n, lambda s, w=csv.write: w(s+"\n"))
            end=time.time()
            print n+' '+str(end-start)+' seconds'
            csv.close()
        except:
            print n
            os.remove(completeName)

In [6]:
import sys
from ftplib import FTP
import time
import os

ftp=FTP("218.210.127.247")
ftp.login("root","cccccccc")
path='/Volumes/Untitled 1'

start=time.time()
retrieveData(ftp, tryList, path)
end=time.time()
print 'process time '+str(end-start)+' seconds'

NameError: name 'tryList' is not defined

In [ ]:
import sys
from ftplib import FTP
import time
import os

ftp=FTP("218.210.127.247")
ftp.login("root","cccccccc")
savepath='/Volumes/Untitled 1'

start=time.time()
retrieveData(ftp, downloadList, savepath)
end=time.time()

print 'process time '+str(end-start)+' seconds'

2015-08-19-10-19-14_131072.csv 13.4010038376 seconds
2015-08-19-10-30-11_131072.csv 13.0404310226 seconds
2015-08-19-10-41-06_131072.csv 14.1442232132 seconds
2015-08-19-10-52-02_131072.csv 13.209856987 seconds
2015-08-19-11-02-59_131072.csv 14.0997128487 seconds
2015-08-19-11-13-54_131072.csv 14.1143388748 seconds
2015-08-19-11-24-50_131072.csv 14.6655349731 seconds
2015-08-19-11-35-46_131072.csv 13.8130450249 seconds
2015-08-19-11-46-42_131072.csv 15.1222288609 seconds
2015-08-19-11-57-37_131072.csv 15.0729420185 seconds
2015-08-19-12-08-34_131072.csv 16.0566220284 seconds
2015-08-19-12-19-29_131072.csv 14.3175961971 seconds
2015-08-19-12-30-26_131072.csv 13.7975208759 seconds
2015-08-19-12-41-22_131072.csv 14.5259661674 seconds
2015-08-19-12-52-17_131072.csv 15.6754882336 seconds
2015-08-19-13-03-13_131072.csv 15.8615458012 seconds
2015-08-19-13-14-09_131072.csv 16.1048529148 seconds
2015-08-19-13-25-05_131072.csv 14.1487309933 seconds
2015-08-19-13-36-01_131072.csv 15.5027689934 se

In [30]:
cvs=open('2015-07-01-00-00-27_131072.csv','w')

In [8]:
import pip

def install(package):
    pip.main(['install', package])

# Example
if __name__ == '__main__':
    install('pandas')

Downloading/unpacking pandas
Downloading/unpacking pandas
  Running setup.py (path:/tmp/pip_build_vagrant/pandas/setup.py) egg_info for package pandas
  Running setup.py (path:/tmp/pip_build_vagrant/pandas/setup.py) egg_info for package pandas
    
    
    no previously-included directories found matching 'doc/build'
    no previously-included directories found matching 'doc/build'
  Running setup.py install for pandas
  Running setup.py install for pandas
    building 'pandas.index' extension
    building 'pandas.index' extension
    i686-linux-gnu-gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC -Ipandas/src/klib -Ipandas/src -I/usr/lib/python2.7/dist-packages/numpy/core/include -I/usr/include/python2.7 -c pandas/index.c -o build/temp.linux-i686-2.7/pandas/index.o
    i686-linux-gnu-gcc -pthread -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fPIC -Ipandas/src/klib -Ipandas/src -I/usr/lib/python2.7/dist-packages/numpy/

In [10]:
import Pandas as pd
import numpy as np
df = pd.read_csv("2015-03-23-18-14-03_131072.csv")# read the file in the same folder
df.head(10)# print the first 10 entries

ImportError: No module named Pandas

In [28]:
import string
txt=open('2015-03-23-18-14-03_131072.csv','r')
interestedChannel=(1,4)

def extractLineSplit(line,ch):
    splitedLine=string.split(line,',')
    output=[]
    for c in ch:
        output.append(float(splitedLine[c-1]))
    return output

start=time.time()
ch1,ch4=[],[]
lines=[]
for line in txt:
    lines.append(line)
txt.close
for line in lines[1:]:
    temp = extractLineSplit(line,interestedChannel)
    ch1.append(temp[0])
    ch4.append(temp[1])
end=time.time()

print len(ch1)
print len(ch4)
print 'process time '+str(end-start)+' seconds/per file'

131072
131072
process time 0.26841711998 seconds/per file


In [ ]:
ch1RDD,ch4RDD=sc.parallelize(ch1,4),sc.parallelize(ch4,4)
def nomalize(RDD):
    m=RDD.mean()
    normalizedRDD=RDD.map(lambda x:x-m)
    return 

In [ ]:
def EEMD(RDD):
    IMFs=[]
    
    return IMF

def cubeSpine():
    
    
    return cubeSpineLine



In [15]:
filenameRDD=sc.parallelize(cleanedDirList,4)
print filenameRDD.take(4)


['2015-01-18-10-53-45_131072.csv', '2015-01-18-11-04-40_131072.csv', '2015-01-18-11-15-35_131072.csv', '2015-01-18-11-37-27_131072.csv']


In [18]:
testlist=[]
help(testlist.write)

AttributeError: 'list' object has no attribute 'write'

####Now we define file retrieving protocal.

####The demo version, howerer, will only execute data in a limited scoup. 

###__Step 2. Defining Data Spark Translations__
####


In [24]:
print(len(ftpDirList))
print()

33992


###__Step 3. Setting Off Spark Actions__
####[FFT Pipeline](https://highlyscalable.wordpress.com/2013/08/20/in-stream-big-data-processing/)

###__Step 4. Data Visualizing__
####

###__Step 5. Exporting Results__
####One may notice that all executed data 

##Step ?. Accessing Central Weather Bureau EQ Database
###The Central Weather Bureau offers a 

#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)


####[Progress Bar](http://nbviewer.ipython.org/github/johnkabler/ipython/blob/1.x/examples/notebooks/Progress%20Bars.ipynb)

In [8]:
import uuid
import time
from IPython.display import HTML, Javascript, display
divid = str(uuid.uuid4())
pb = HTML(
"""
<div style="border: 1px solid black; width:500px">
  <div id="%s" style="background-color:blue; width:0%%">&nbsp;</div>
</div> 
""" % divid)
display(pb)
for i in range(1,101):
    time.sleep(0.1)
    display(Javascript("$('div#%s').width('%i%%')" % (divid, i)))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
from __future__ import print_function
import sys, time

class ProgressBar:
    def __init__(self, iterations):
        self.iterations = iterations
        self.prog_bar = '[]'
        self.fill_char = '*'
        self.width = 50
        self.__update_amount(0)
    def animate(self, iter):
        print('\r', self, end='')
        sys.stdout.flush()
        self.update_iteration(iter + 1)
    def update_iteration(self, elapsed_iter):
        self.__update_amount((elapsed_iter / float(self.iterations)) * 100.0)
        self.prog_bar += '  %d of %s complete' % (elapsed_iter, self.iterations)
    def __update_amount(self, new_amount):
        percent_done = int(round((new_amount / 100.0) * 100.0))
        all_full = self.width - 2
        num_hashes = int(round((percent_done / 100.0) * all_full))
        self.prog_bar = '[' + self.fill_char * num_hashes + ' ' * (all_full - num_hashes) + ']'
        pct_place = (len(self.prog_bar) // 2) - len(str(percent_done))
        pct_string = '%d%%' % percent_done
        self.prog_bar = self.prog_bar[0:pct_place] + \
            (pct_string + self.prog_bar[pct_place + len(pct_string):])
    def __str__(self):
        return str(self.prog_bar)

In [10]:
p = ProgressBar(1000)
for i in range(1001):
    time.sleep(0.005)
    p.animate(i)

 [*********************100%***********************]  1000 of 1000 complete